## Question  5: Active contours
Similar to the previous problems, code of the functions are written in
`q5_funcs.py` and main code of the problem is in `q5.py`

### q5_funcs

First function takes a contour ( contour is a list of points ) and an integer ratio, and expands contour by
adding "ratio" number of points between each two consecutive points:

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import ffmpeg


def expand_contour(contour, ratio):
    result = []
    for i, point in enumerate(contour):
        point1 = contour[i]
        x1, y1 = point1[0], point1[1]
        print("point 1:", x1, y1)
        point2 = contour[i - 1]
        x2, y2 = point2[0], point2[1]
        print("point 2:", x2, y2)
        r, s = int((x1 - x2) / ratio), int((y1 - y2) / ratio)
        print("r,s:", r, s)
        if r == 0 and s == 0:
            result.append(point1)
            continue
        for j in range(ratio):
            point3 = [x2 + j * r, y2 + j * s]
            print("point 3:", point3)
            result.append(point3)
        result.append(point1)
        print("#############")
    return result


Function `draw_contour` takes an image, a contour and a color as input and draws a contour with the given color on the image:

In [ ]:
def draw_contour(img, contour, color):
    new_contour = np.roll(contour, 1, axis=1)
    result = cv2.polylines(img.copy(), [new_contour], True, color, 1)
    return result


Function `get_image_grads` outputs an array containing the gradient at each pixel of the image

In [ ]:
def get_image_grads(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    x_derivative = img_gray - np.roll(img_gray, 1, axis=0)
    y_derivative = img_gray - np.roll(img_gray, 1, axis=1)
    result = -np.sqrt(np.square(x_derivative) + np.square(y_derivative))
    return result


Next four functions are used to compute energies used in active contours.
Their implementation is simple and based on the formulas for energies, so we don't explain them seprately:

In [ ]:
def calculate_external_energy(img_grads, contour):
    e_external = 0
    for point in contour:
        e_external += img_grads[point[0], point[1]]
    return -e_external


def calculate_average_distance(contour):
    shifted_contour = np.roll(contour, 1, axis=0)
    d = np.average(np.sum(np.square(contour - shifted_contour), axis=1))
    return d, shifted_contour


def calculate_curvature(contour, shifted_contour):
    shifted_contour2 = np.roll(shifted_contour, 1, axis=0)
    matrix = contour - 2 * shifted_contour + shifted_contour2
    d = np.sum(np.square(matrix))
    return d


def calculate_internal_energy(contour, alpha, beta):
    d, shifted_contour = calculate_average_distance(contour)
    curvature = calculate_curvature(contour, shifted_contour)
    e_internal = alpha * np.sum(np.square(np.sum(np.square(contour - shifted_contour), axis=1) - d)) + beta * curvature
    return e_internal


def calculate_total_energy(img_grads, contour, alpha, beta, landa):
    e_external = calculate_external_energy(img_grads, contour)
    e_internal = calculate_internal_energy(contour, alpha, beta)
    return e_internal + landa * e_external


### Updating contour
#### Method 1: Greedy algorithm
first method for updating contour is greedy algorithm, which performs greedy search for each point in contour and finds the
best point for each one :

In [ ]:
def update_contour_greedy(img_grads, contour, alpha, beta, landa, search_radius):
    result, temp = contour.copy(), contour.copy()
    e = calculate_total_energy(img_grads, result, alpha, beta, landa)
    for i, point in enumerate(contour):
        temp = result.copy()
        for j in range(-search_radius, search_radius + 1):
            for k in range(-search_radius, search_radius + 1):
                temp[i] = [temp[i, 0] + j, temp[i, 1] + k]
                e1 = calculate_total_energy(img_grads, temp, alpha, beta, landa)
                if e1 < e:
                    result[i] = temp[i]
                    e = calculate_total_energy(img_grads, result, alpha, beta, landa)
    return result

Now we can use above function to greedily apply active contours on an image :
( This function is not complete, since we don't use greedy algorithm for the actual problem )

In [ ]:
def active_contours_greedy(img, init_contour, alpha, beta, landa):
    img_grads = get_image_grads(img)
    initial_energy = calculate_total_energy(img_grads, init_contour, alpha, beta, landa)
    contour = init_contour.copy()
    for i in range(10):
        contour = update_contour_greedy(img_grads, contour, alpha, beta, landa, 1)
        plt.imshow(draw_contour(img, contour, (0, 255, 0)))
        plt.show()
    return contour

#### Method 2: Dynamic Programming
From here, we have functions ued for updating contour and performing
active contours using DP


The first two functions are used to calculate energy between two consecutive points ( which is needed in DP)
and finding the closest neighbor to a point, from the neighborhood of its previous point, and output its energy and index:

In [ ]:
def calculate_energy_between_points(point1, point2, img_grads, d, alpha, landa):
    e1 = -landa * img_grads[point2[0], point2[1]]
    e2 = alpha * np.square(np.sum(np.square(point1 - point2) - d))
    return e1 + e2


def find_closest_neighbor(current_points, previous_point, energies, i, search_radius, img_grads, d, alpha, landa):
    index, energy = 0, float('inf')
    t = 0
    for j in range(-search_radius, search_radius + 1):
        for k in range(-search_radius, search_radius + 1):
            point = np.asarray([previous_point[0] + j, previous_point[1] + k])
            e = calculate_energy_between_points(current_points, point, img_grads, d, alpha, landa) + energies[
                i - 1, t]
            if e < energy:
                index = t
                energy = e
            t += 1
    return index, energy

Function `get_indices_energies_dp` performs DP on a given contour and outputs two n*m matrices: one containing the indices, which show
the index of the point that each point corresponds to , and other containing the energies :

In [ ]:
def get_indices_energies_dp(img_grads, contour, alpha, landa, search_radius):
    m = (2 * search_radius + 1) ** 2
    n = float('inf')
    result_indices, result_energies = np.zeros((contour.shape[0], m)), np.zeros((contour.shape[0], m))
    d, shifted_contour = calculate_average_distance(contour)

    for r in range(m):  # iterating over first point's neighborhood
        temp_indices, temp_energies = np.zeros((contour.shape[0], m)), np.zeros((contour.shape[0], m))
        x, y = -1 + int(r / (2 * search_radius + 1)), -1 + (r % (2 * search_radius + 1))
        point = np.asarray([contour[0, 0] + x, contour[0, 1] + y])

        for i in range(m):
            temp_indices[1, i] = r
            point2 = np.asarray([contour[1, 0] + x, contour[1, 1] + y])
            temp_energies[1, i] = calculate_energy_between_points(point, point2, img_grads, d, alpha, landa)

        for i in range(2, contour.shape[0]):  # iterating over contour points
            t = 0
            for j in range(-search_radius, search_radius + 1):
                for k in range(-search_radius, search_radius + 1):
                    current_point = np.asarray([contour[i, 0] + j, contour[i, 1] + k])
                    previous_point = contour[i - 1].reshape(contour.shape[1], 1)
                    next_index, next_energy = find_closest_neighbor(current_point,
                                                                    previous_point,
                                                                    temp_energies, i, search_radius, img_grads, d,
                                                                    alpha,
                                                                    landa)
                    temp_indices[i, t] = next_index
                    temp_energies[i, t] = next_energy
                    t += 1
        previous_point = contour[-1].reshape(contour.shape[1], 1)
        index, energy = find_closest_neighbor(point, previous_point, temp_energies, contour.shape[0] - 1, search_radius,
                                              img_grads, d, alpha, landa)

        if energy < n:
            n = energy
            temp_energies[0, r] = energy
            temp_indices[0, r] = index
            result_energies = temp_energies.copy()
            result_indices = temp_indices.copy()

    return result_indices, result_energies

Function `update_contour_dp` uses previous functions to update contour using DP :

In [ ]:
def update_contour_dp(contour, indices, search_radius):
    result = np.zeros(contour.shape, dtype='int')
    index = int(indices[1, 0])
    index_n = int(indices[0, index])
    x, y = -1 + int(index_n / (2 * search_radius + 1)), -1 + (index_n % (2 * search_radius + 1))
    result[-1, 0], result[-1, 1] = contour[-1, 0] + x, contour[-1, 1] + y
    for i in reversed(range(1, contour.shape[0])):
        index_n = int(indices[i, index_n])
        x, y = -1 + int(index_n / (2 * search_radius + 1)), -1 + (index_n % (2 * search_radius + 1))
        result[i - 1, 0], result[i - 1, 1] = int(contour[i - 1, 0] + x), int(contour[i - 1, 1] + y)
    return result


Finally, function `active_contours_dp` performs active contours on an image and an initial contour using DP.
Function outputs a list of images which are the result of drawing contour on the original image in each iteration.

In [ ]:
def active_contours_dp(img, init_contour, alpha, beta, landa, num_of_iter=10):
    img_grads = get_image_grads(img)
    results = []
    initial_energy = calculate_total_energy(img_grads, init_contour, alpha, beta, landa)
    contour = init_contour.copy()
    result = draw_contour(img, contour, (0, 0, 0))
    results.append(result)

    for i in range(num_of_iter):
        indices, energies = get_indices_energies_dp(img_grads, contour, alpha, landa, 1)
        contour = update_contour_dp(contour, indices, 1).astype('int')
        result = draw_contour(img, contour, (0, 0, 0))
        results.append(result)
        # plt.imshow(draw_contour(img, contour, (0, 255, 0)))
        # plt.show()

    return results

Function `generate_video` takes resulting frames, output path , frame size and fps and outputs a
video created using the given parameters :

In [ ]:
def generate_video(results, output_path, size, fps):
    video = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for img in results:
        video.write(img)
    video.release()



### q5.py
Now, we will use the functions implemented previously.

First, we take initial contour points from user :

In [ ]:
import q5_funcs
import matplotlib.pyplot as plt
import numpy as np
import cv2

input_path = "inputs/tasbih.jpg"

img = cv2.imread(input_path)

contour = []


def mouse_clicked(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        contour.append([y, x])


cv2.imshow('img', img)
cv2.setMouseCallback('img', mouse_clicked)
cv2.waitKey()


Then, we expand points in contour and apply active contours, get the frames and save the video :

In [ ]:
contour = q5_funcs.expand_contour(contour, 20)
contour = np.asarray(contour, dtype='int32')
print(contour.shape)
results = q5_funcs.active_contours_dp(img, contour, 0.0001, 1, 1, 15)
q5_funcs.generate_video(results, "outputs/video.mp4", (img.shape[1], img.shape[0]), 3)
